In [2]:
import numpy as np
    
def generateBesselBA(vol_params = None,psf_params = None): 
    # Uout2 = generateBesselBA(vol_params,psf_params)
    
    # This function generates a Bessel back aperture intensity profile with
# the specified aberrations applied. The inputs are
    
    # - vol_params          - Struct with parameters for the volume generation
#          .vol_sz      - 3-element vector with the size (in um) of the
#                         volume to generate (default = 100x100x30um)
#          .vres        - resolution to simulate volume at (default = 2
#                         samples/um)
# - psf_params          - Struct contaning the parameters for the PSF
#          .NA          - Numerical aperture of Gaussian beam
#          .objNA       - Numerical aperture of objective lens
#          .n           - Refractive index of propagation material
#          .lambda      - Two-photon excitation wavelength (um)
#          .obj_fl      - Objective focal length (mm)
#          .ss          - Subsampling factor for fresnel propagation
#          .sampling    - Spatial sampling for tissue occlusion mask
#          .zernikeDst  - Microscope aberration weights (Zernike basis) as
#                         a function of position
# The output is
# - Uout2               - Output scalar field
    
    # 2017 - Alex Song
    
    ###########################################################################
    
    fl = single(psf_params.obj_fl / 1000)
    
    D2 = single(1e-06 * (1 / vol_params.vres) / psf_params.ss)
    
    N = single(1e-06 * (vol_params.vasc_sz(np.arange(1,2+1)) - vol_params.vol_sz(np.arange(1,2+1))) / D2)
    
    D1 = single(np.amax(gaussianBeamSize(psf_params,fl * 1000000.0) / 1000000.0) / np.amin(N))
    
    nre = single(psf_params.n)
    
    rad = single(np.tan(np.arcsin(psf_params.NA / nre)) * fl)
    
    objrad = single(np.tan(np.arcsin(psf_params.objNA / nre)) * fl)
    
    k = 2 * nre * np.pi / single(psf_params.vlambda * 1e-06)
    
    X,Y = np.meshgrid((np.arange(- N(1) / 2,N(1) / 2 - 1+1)) * D1,(np.arange(- N(2) / 2,N(2) / 2 - 1+1)) * D1)
    width = 1.4e-10 * np.tan(np.arcsin(psf_params.NA / nre)) / (fl * 1e-06 * 2 * psf_params.length)
    if (not isfield(psf_params,'bprof')  or len(psf_params.bprof)==0):
        Uout = generateBesselProfile(X,Y,rad,width,k,fl)
    else:
        Uout = generateBesselProfile(X,Y,rad,width,k,fl,psf_params.bprof)
    
    imax = np.round(vol_params.vol_sz(1) / psf_params.sampling) + 1
    jmax = np.round(vol_params.vol_sz(2) / psf_params.sampling) + 1
    if (imax * jmax == 1 or (not isfield(psf_params,'zernikeDst') ) or len(psf_params.zernikeDst)==0):
        abb = generateZernike(psf_params)
        Uout2 = applyZernike(Uout,X / objrad,Y / objrad,k,abb)
    else:
        Uout2 = np.zeros(imax,jmax)
        for i in np.arange(1,imax+1).reshape(-1):
            for j in np.arange(1,jmax+1).reshape(-1):
                abb = generateZernike(psf_params)
                Uout2[i,j] = applyZernike(Uout,X,Y,k,abb)
    
    return Uout2

C:\Users\ryant\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ryant\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\ryant\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
